# Income analysis for Denmark

In this data project we analyize the income charateristics in Denmark. We focus our analysis on income for different age groups, regions and sectors in Denmark from 2013-2022.

First we import the data using and API from Danmarks Statistik. We use two statistics, LONS50 and LONS30, clean them both and merge them. 

We then start our analysis by first examining the overall income devlopment from 2013 to 2022, looking at the hourly wage (earnings in DKK per hour worked) for all of Denmark. 

We then analyize the income devlopment and income distrubution for the different areas in Denmark using a box-plot. We then look at the income distrubution for the different age groups - also using a box-plot.

Lastly we examine the devlopment and growth rate in the income for different sector. Using an interactive graph and box we also examine the sector income development for different areas. We finish by conluding our analysis of income characteristics in Denmark.

First we install the necessary programs:

In [4]:
# The DST API wrapper
#%pip install git+https://github.com/alemartinello/dstapi

# A wrapper for multiple APIs with a pandas interface
#%pip install pandas-datareader

# For data visualization
#%pip install seaborn

# For widgets
%conda install -c conda-forge nodejs
%pip install ipywidgets

# Enable the necessary Jupyter notebook extensions
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter nbextension install --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager


Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console deja

The we write necessary imports and set magics:

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import warnings
import seaborn as sns
import plotly.express as px
from IPython.display import display

import datetime
import pandas_datareader 
from dstapi import DstApi

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# Importing python file
import dataprojecteksamen

#We use this later on for our plots
warnings.filterwarnings('ignore', category=FutureWarning)


# Read and clean data

Importing the first data set about income where one of the variables is age from DST through an API (we will refer to the data set as income-age).

In [ ]:
ind = DstApi('LONS50') 

Getting a quick overview over the data

In [ ]:
tabsum = ind.tablesummary(language='en')
display(tabsum)

Then we would like to see the available values for a each variable and their id: 

In [ ]:
# The available values for a each variable and their id: 
for variable in tabsum['variable name']:
   print(variable+':')
   display(ind.variable_levels(variable, language='en'))


Then we use the define_base_params -method to give us a nice template (where from we can select available data).

In [ ]:
params = ind._define_base_params(language='en')
params

We then choose the data we want as our parameters. We want to examine the income for different age groups and sektors.

In [ ]:
# The data we want as our parameters 
params = {'table': 'lons50',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'ALDER1', 'values': ['20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-']},
  {'code': 'SEKTOR', 'values': ['1018', '1016', '1046']},
  {'code': 'AFLOEN', 'values': ['TIFA']},
  {'code': 'LONGRP', 'values': ['LTOT']},
  {'code': 'LØNMÅL', 'values': ['FORINKL']},
  {'code': 'KØN', 'values': ['MOK']},
  {'code': 'Tid', 'values': ['*']}]}


To get an overview of the data we print some infomation and the top of the table showing the data. Remove the # to show the dataframe.

In [ ]:
inc_api = ind.get_data(params=params)
#inc_api.info()
#inc_api.head()

Next, we sort the data and again print the top of the table.

In [ ]:
inc_api.sort_values(by=['ALDER1', 'TID', 'SEKTOR'], inplace=True)
inc_api.head(5)

It seems that the "INDHOLD" column is currently stored as an object data type and the "TID" column is stored as an int64, so we will convert it to a float data type.

In [ ]:
inc_api['INDHOLD'] = inc_api['INDHOLD'].astype(float)
#print(inc_api['INDHOLD'].dtype)

In [ ]:
inc_api['TID'] = inc_api['TID'].astype(float)
#print(inc_api['TID'].dtype)

We import the second data set about income where one of the variables is area in Denmark from DST through an API (we will refer to the data set as income-area). We will for this income-area data set go through the same methoed as we did for the income-age data set. 

In [ ]:
ind1 = DstApi('LONS30') 

Getting a quick overview over the data

In [ ]:
tabsum1 = ind1.tablesummary(language='en')
display(tabsum1)

Then we would like to see the available values for a each variable and their id: 

In [ ]:
#for variable in tabsum1['variable name']:
  #print(variable+':')
  #display(ind.variable_levels(variable, language='en'))

Then we use the define_base_params -method to give us a nice template (where from we can select available data).

In [ ]:
params1 = ind1._define_base_params(language='en')
params1

Here we choose the data we want as our parameters:

In [ ]:
params1 = {'table': 'lons30',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'SEKTOR', 'values': ['1018', '1016', '1046']},
  {'code': 'AFLOEN', 'values': ['TIFA']},
  {'code': 'LONGRP', 'values': ['LTOT']},
  {'code': 'LØNMÅL', 'values': ['FORINKL']},
  {'code': 'KØN', 'values': ['MOK']},
  {'code': 'Tid', 'values': ['*']}]}

To get an overview of the data we print some infomation and the top of the table showing the data. Remove the # to show the dataframe.

In [ ]:
inc1_api = ind1.get_data(params=params1)
#inc1_api.info()
#inc1_api.head()

Next, we sort the data and again print the top of the table:

In [ ]:
inc1_api.sort_values(by=['OMRÅDE', 'TID', 'SEKTOR'], inplace=True)
#inc1_api.head(5)

Importing the second data set about income where one of the variables is area in Denmark from DST through an API (we will refer to the data set as income-area). We will for this income-area data set go through the same methoed as we did for the income-age data set. 

In [ ]:
inc1_api['INDHOLD'] = inc1_api['INDHOLD'].astype(float)
#print(inc1_api['INDHOLD'].dtype)

In [ ]:
inc1_api['TID'] = inc1_api['TID'].astype(float)
#print(inc1_api['TID'].dtype)

# Cleaning the data sets

We now clean the data set by dropping irrelevant variables and removing duplicates.

Dropping irrelevant variables from first data set:

In [ ]:
Inc_age = inc_api.drop(['LONGRP', 'LØNMÅL', 'AFLOEN','KØN'],axis=1) 
Inc_age.head(10)

Dropping irrelevant variables from second data set:

In [ ]:
Inc_area = inc1_api.drop(['LONGRP', 'LØNMÅL', 'AFLOEN','KØN'],axis=1) # Drop irrelevant variables
Inc_area.head(10)

Cleaning the data by removing duplicates:

In [ ]:
Inc_age_cleaned = Inc_age.dropna().drop_duplicates()
Inc_area_cleaned = Inc_area.dropna().drop_duplicates()

# Merge data sets

We now merge the two data set into one.

We merge datasets on common 'TID' and 'SEKTOR'. We are using an inner join here, meaning we only keep rows that match in both datasets:

In [ ]:
merged_df = pd.merge(Inc_age_cleaned, Inc_area_cleaned, on=['TID', 'SEKTOR'], suffixes=('_age', '_area'))

# Check the merged data
print(merged_df.head())

# Analysis

We start our analysis with a simpel overview over the income development in the years 2013-2022. The income (measured as the hourly wage) has been increasing every year in the period.

In [ ]:
# We group the data by year and calculate the average income for each year
avg_income_by_year = merged_df.groupby('TID')['INDHOLD_age'].mean()

# We import the plot function from the dataproject python file
from dataprojecteksamen import plot_avg_income_over_years

# Then plot the function
plot_avg_income_over_years(avg_income_by_year)

We see that the average income in Denmark has been increasing over the entire periode from 2014 to 2022.

Next, we show the development in the income for the different areas in Denmark for the same period.

In [ ]:
# We import the plot function from the python file
from dataprojecteksamen import plot_avg_income_by_year_area

# Plot the function
plot_avg_income_by_year_area(merged_df)

Here we see a much higher income in Region Hovedstaden compared to the other areas, and Region Syddanmark has the lowest average income. The average income has been increasin over the entire period for all areas.

We then want to examine the income distrubution for different age groups and areas for the most recent year. In the next figure, we present two boxplots where the first one shows the income distribution by age for 2022 and the second shows the income distribution by area for 2022.

We find the most recent year in the merged dataset:

In [ ]:
most_recent_year = int(merged_df['TID'].max())
print(f"Most recent year in the dataset: {most_recent_year}")

We plot the income distrbution by area in the year 2022:

In [ ]:
# We import the plot functions
from dataprojecteksamen import plot_income_distribution_by_area

# We define the most recent year
most_recent_year = 2022

# We plot the functions
plot_income_distribution_by_area(merged_df, most_recent_year)

The line indicates the median income and the dot shows the mean income. 
We see that the median income is largest in Region Hovedstaden. We also see that the median is higher than the mean, which suggests that most people have higher incomes, but there are a few individuals with very low incomes that are affecting the average. For all other regions the median is lower than the mean, indicating that a smaller portion of the population in these areas contribute to raising the average income. Overall, we see that the income in Region Hovedstaden pulls up the average income for all of Denmark. The lowest average and median income is in Region Nordjylland. 

We then plot the income distrbution by age group in the year 2022:

In [ ]:
# We import the plot functions
from dataprojecteksamen import plot_income_distribution_by_age

# We define the most recent year
most_recent_year = 2022

# We plot the functions
plot_income_distribution_by_age(merged_df, most_recent_year)

We see that the average income increases from age group 20-24 all the way to 50-54 years whereafter it falls. This is most likely because young people in the age group 20-24 are studying and therefore, and first in the age group 25-29 start to enter the workforce. As experience increase pay most likely also icnreases. In the age group 55-59 people start to enter retirement, which may contribute to the fall in average income.

We see from age group 25-29 to 55-59 the median is higher than the average income indicating that few individuals contribute to dragging down the average. Whereas for the group 20-24 and 60 years and over the median is slighty higher than the average, which indicates that the distrubtion is overall fairly equal for these groups however a few indivuals contribute a slightly higher average.

Last we want to examine the average income development for different sectors in Denmark.

We plot the average income for the different sectors for the periode 2013-2022 for different areas. The area can be choosen using the dropdown box.

In [ ]:
# Import the function from the python file
from dataprojecteksamen import interactive_plot

# Plot the function
interactive_plot(Inc_age_cleaned, Inc_area_cleaned)

When looking at all of Denmark we see that the income is greates over the entire period in the sector Corporations and organizations and lowest in Municipal and regional goverment, but has been increasing for all the three sectors over the entire period. 

In Region Hovedstaden the average income is much lower in the sector Municipal and regional goverment compared to the other two, however the average income from sector for Goverment including social security funds have increased in 2022 to be almost as high as Cooperations and organizations. 

This picture is very different when we look at the other areas in Denmark. In all other regions the income is greates in the sector for Goverment including social security funds. And in Region Nordjylland and Sjælland it is the lowest. The plots indicate that the sector Corporations and organizations provide a higher wage in the regions containing the two largest cities in Denmark, Copenhagen and Aarhus (Region Hovedstaden and Midtjylland). Wheras in the more rural areas the average income stemming from goverment and muncipial and regional goverment sectors is higher than from the private sector. 

We then examine the growth rate from 2013 to 2022 for the different sectors. The drop box can be used to select the areas.

In [ ]:
# We import the growth rate funtion from the python file
from dataprojecteksamen import interactive_growth_rate

# We show the box
interactive_growth_rate(Inc_age_cleaned, Inc_area_cleaned)

We see from the drop box that in all of Denmark the growth rate in average income has been the greatest in the sector Municipal and regional gowerment (which is also the case when looking at the regions indivdiually) and lowest in Corporations and organizations. This indicates a generall greater income growth in the public sector compared to private. 

# Conclusion

We find that income has been increasing in all regions in Denmark from 2013 to 2022. The income is greatest in the region 'Hovedstaden' and the least in 'Nordjylland' for all years, which most likely is because of 'Hovedstaden' including the capital of Denmark. Generally, we would expect to see more higher-salary jobs in capitals and larger citites. We also show two box plot with the median income for all regions and age groups in 2022. We see that although the median income for Copenhagen is very high, there is a large variability in the income distrubution for the bottom 50 pct. of income earners in 2022, with few indivduals lowering the average.

In 2022 we see that the income median is increasing in age groups but peaks at the age group 50-54 whereafter it falls. This is presumably because young people are studying and earning a lower wage for the first years of their careers whereafter salaries increase with more experience in the job market/senior positions. The fall in income in the older age groups is most likely caused by more people retiring and switching to pensions that provides a lower income than working. For almost all age groups the median is relativly high in the box-plot, which illustrates that there is greater income variability in the 50 pct. lowest earners compared to the 50 pct. highest, and that a smaller portion of the groups lowers the average.

The average income has increased for all sectors in Denmark from 2013 to 2022, however the sector Municipal and regional gowerment has had the greates growth rate, which is also the case when looking at the individual areas. In region Hovedstaden the highest income is from the sector Corporations and organizations, wheras in the other areas the sector for Goverment including social security funds has the highest average income.